In [1]:
import os

In [2]:
os.environ["PYTHONPATH"] = "../my_nanochat"

He has what he calls a "poor man's configurator" in `configurator.py` which is how he handles passing config values on the command line and via config file. I'll copy that first because I'll need it for base_train.

In [3]:
# he runs it this way so it gets access to argv, expect error here
exec(open("../my_nanochat/my_nanochat/my_configurator.py").read())

AssertionError: TODO implement config file support

See how argument overriding works:

In [5]:
!python -m scripts.my_base_train

user_config: {'device_type': '', 'depth': 20, 'max_seq_len': 2048, 'num_iterations': -1, 'device_batch_size': 32, 'total_batch_size': 524288, 'embedding_lr': 0.2, 'unembedding_lr': 0.004, 'weight_decay': 0.0, 'matrix_lr': 0.02, 'grad_clip': 1.0, 'warmup_ratio': 0.0, 'warmdown_ratio': 0.2, 'final_lr_frac': 0.0, 'eval_every': 250, 'eval_tokens': 10485760, 'core_metric_every': 2000, 'core_metric_max_per_task': 500, 'sample_every': 2000, 'model_tag': ''}


In [6]:
!python -m scripts.my_base_train --depth=4

overriding depth = 4
user_config: {'device_type': '', 'depth': 4, 'max_seq_len': 2048, 'num_iterations': -1, 'device_batch_size': 32, 'total_batch_size': 524288, 'embedding_lr': 0.2, 'unembedding_lr': 0.004, 'weight_decay': 0.0, 'matrix_lr': 0.02, 'grad_clip': 1.0, 'warmup_ratio': 0.0, 'warmdown_ratio': 0.2, 'final_lr_frac': 0.0, 'eval_every': 250, 'eval_tokens': 10485760, 'core_metric_every': 2000, 'core_metric_max_per_task': 500, 'sample_every': 2000, 'model_tag': ''}


Do a tiny training:

In [9]:
!python -m scripts.my_base_train \
    --depth=4 \
    --max_seq_len=128 \
    --device_batch_size=1 \
    --num_iterations=10 \
    --total_batch_size=128

overriding depth = 4
overriding max_seq_len = 128
overriding device_batch_size = 1
overriding num_iterations = 10
overriding total_batch_size = 128
user_config: {'device_type': '', 'depth': 4, 'max_seq_len': 128, 'num_iterations': 10, 'device_batch_size': 1, 'total_batch_size': 128, 'embedding_lr': 0.2, 'unembedding_lr': 0.004, 'weight_decay': 0.0, 'matrix_lr': 0.02, 'grad_clip': 1.0, 'warmup_ratio': 0.0, 'warmdown_ratio': 0.2, 'final_lr_frac': 0.0, 'eval_every': 250, 'eval_tokens': 10485760, 'core_metric_every': 2000, 'core_metric_max_per_task': 500, 'sample_every': 2000, 'model_tag': ''}
Autodetected device type: mps
Vocab size: 65,537
num_layers: 4
model_dim: 256
num_heads: 2
num_kv_heads: 2
Tokens / micro-batch / rank: 1 x 128 = 128
Tokens / micro-batch: 128
Total batch size 128 => gradient accumulation steps: 1
GPT(
  (transformer): ModuleDict(
    (wte): Embedding(65537, 256)
    (h): ModuleList(
      (0-3): 4 x Block(
        (attn): CausalSelfAttention(
          (c_q): Linear

In [10]:
!ls -lh ~/.cache/my_nanochat/base_checkpoints/d4

total 852232
-rw-r--r--  1 ericsilberstein  staff   767B Nov 11 11:19 meta_000010.json
-rw-r--r--  1 ericsilberstein  staff   140M Nov 11 11:19 model_000010.pt
-rw-r--r--  1 ericsilberstein  staff   268M Nov 11 11:19 optim_000010.pt


What I did as part of this challenge:

- Created `scripts/my_base_train.py` but without the model evaluations, sampling, and some of the logging
- Added `compute_init()` and `compute_cleanup()` to `my_common.py`. These will mostly apply later when we get to DDP, although setting a random seed applies now.
- Created `checkpoint_manager.py` with `save_checkpoint()` but no other functions yet
- Created `my_configurator.py` which `my_base_train.py` uses and I assume other scripts will alos u